### Analysing masters present in bug reports

In [1]:
!pip install networkx

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import _pickle as pickle

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
MAX_SEQUENCE_LENGTH_T = 20 # 20
MAX_SEQUENCE_LENGTH_D = 20 # 80
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

'''
    Configuration
'''
epochs = 100
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

In [34]:
# Domain to use
DOMAIN = 'openoffice'
'''
    propose_centroid_bert_
    propose_bert_triplet_
    propose_bert_
    deepQL_trainable_
    deepQL_no_trainable_
    baseline_dwen_
    baseline_
'''
METHOD = 'deepQL_trainable_{}'.format(epochs)
EMBED_METHOD='bert'
TOKEN = 'bert'
ONLY_BUCKETS = False
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [6]:
path_buckets = 'data/normalized/{}/{}.csv'.format(DOMAIN, DOMAIN)

df = pd.read_csv(path_buckets)

In [7]:
df.head()

,bug_id,bug_severity,bug_status,component,creation_ts,delta_ts,description,dup_id,priority,product,resolution,short_desc,version
0,14,trivial,CLOSED,Website general issues,2000-10-17 19:40:00 +0000,2006-02-07 22:23:55 +0000,it would be nice if the combination of OpenOff...,[],P3,Infrastructure,FIXED,openoffice.org issuezillla URL's display a hor...,current
1,15,trivial,CLOSED,Bugzilla,2000-10-19 11:49:00 +0000,2003-12-27 10:23:17 +0000,this task is just a test\nthis task is just at...,[],P5,Infrastructure,FIXED,test task,current
2,16,trivial,CLOSED,_openoffice.org administrative interface (obso...,2000-10-20 15:06:00 +0000,2003-12-06 14:52:32 +0000,Using the ADMIN link on the main page leads to...,[],P1,Infrastructure,FIXED,Administrative Interface not available,current
3,17,trivial,CLOSED,_openoffice.org CVS (obsolete),2000-10-20 18:46:00 +0000,2003-12-06 14:52:32 +0000,Entered into Issuezilla for tracking purposes....,[],P3,Infrastructure,FIXED,update modules file.,current
4,19,trivial,CLOSED,definition,2000-10-20 22:54:00 +0000,2007-09-22 22:15:55 +0000,<text:quo-vadis></text:quo-vadis>\n <text:er...,[],P2,xml,FIXED,"text:quo-vadis, text:ergo-sum",605


In [8]:
G=nx.Graph()
for row in df.iterrows():
    bug_id = row[1]['bug_id']
    dup_id = row[1]['dup_id']
    if dup_id == '[]':
        G.add_node(dup_id)
    else:
        G.add_edges_from([(int(bug_id), int(dup_id))])

In [9]:
# for r in nx.connected_components(G):
#     print(r)

In [10]:
# nx.degree(G)

In [11]:
#nx.draw(G)

In [12]:
stats = {}
for r in nx.connected_components(G):
    group_size = len(r)
    if group_size in stats:
        stats[group_size] += 1
    else:
        stats[group_size] = 1

In [13]:
pd.DataFrame(stats, index=[1]).transpose().rename(columns={1 : 'size_group'})\
    .sort_values(by='size_group', ascending=False).reset_index().rename(columns={'index' : 'group'})

,group,size_group
0,2,4412
1,3,1109
2,4,436
3,5,265
4,6,153
5,7,101
6,8,69
7,9,49
8,11,29
9,10,25


In [19]:
def create_bucket(df):
        print("Creating the buckets...")
        buckets = {}
        G=nx.Graph()
        for row in tqdm(df.iterrows()):
            bug_id = row[1]['bug_id']
            dup_id = row[1]['dup_id']
            if dup_id == '[]':
                G.add_node(bug_id)
            else:
                G.add_edges_from([(int(bug_id), int(dup_id))])
        for g in tqdm(nx.connected_components(G)):
            group = set(g)
            for bug in g:
                master = int(bug)
                query = df[df['bug_id'] == master]
                if query.shape[0] <= 0:
                    group.remove(master)
                    master = np.random.choice(list(group), 1)
            buckets[int(master)] = group
        return buckets

In [20]:
buckets = create_bucket(df)

Creating the buckets...


In [21]:
len(buckets)

58572

In [35]:
os.path.join(DIR, TOKEN, DOMAIN + '_buckets.pkl')

'data/processed/openoffice/bert/openoffice_buckets.pkl'

In [37]:
with open(os.path.join(DIR, TOKEN, DOMAIN + '_buckets.pkl'), 'wb') as f:
    pickle.dump(buckets, f)
print("Buckets saved")

Buckets saved
